# Laboratório RAG com LangChain, Groq e Pinecone

Este notebook demonstra a implementação de um sistema Retrieval-Augmented Generation (RAG) utilizando a biblioteca LangChain, modelos de linguagem da Groq e o Pinecone como banco de dados vetorial. O objetivo é responder a perguntas com base em documentos PDF fornecidos, focando em atividades físicas e musculação.

## Configuração do Ambiente

### Importar bibliotecas

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import zipfile
import os
import shutil
import time

### Configuração das Chaves de API

In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

## Carregando a LLM da Groq

In [ ]:
llm = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

## Carregamento e Processamento dos Documentos PDF

In [ ]:
zip_file_path = 'Arquivos.zip'
extracted_folder_path = 'docs'

### Extração dos PDFs do arquivo ZIP

In [ ]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

### Leitura dos Documentos PDF

In [ ]:
documents = []
for filename in os.listdir(extracted_folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(extracted_folder_path, filename)
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

In [ ]:
# Antes de apagar, é uma boa prática verificar se o diretório existe
if os.path.exists(extracted_folder_path):
    print(f"Apagando o diretório e seu conteúdo: {extracted_folder_path}")
    try:
        # A função rmtree() remove o diretório e tudo que está dentro dele
        shutil.rmtree(extracted_folder_path)
        print("Diretório apagado com sucesso.")
    except OSError as e:
        print(f"Erro: {e.strerror}")
else:
    print(f"O diretório '{extracted_folder_path}' não existe.")

## Geração de Embeddings e Armazenamento no Pinecone

### Divisão dos Documentos em Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


In [ ]:
print(f'Total de chunks: {len(docs)}')

In [ ]:
docs[1]

### Inicializar modelo de embeddings (HuggingFace para uso local/gratuito)

In [ ]:

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

### Inicializar Pinecone

In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:
index_name = "rag-musculacao" 
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()] # change if desired

### Deletar e recriar o índice para garantir que a dimensão esteja correta

In [ ]:

if index_name in existing_indexes:
    print(f"Deletando o índice existente '{index_name}'...")
    pc.delete_index(index_name)
    time.sleep(1) # Aguardar a exclusão

In [ ]:
pc.create_index(
    name=index_name,
    metric="cosine",
    dimension=384, # Isso irá retornar 384
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
while not pc.describe_index(index_name).status["ready"]:
    print("Aguardando o índice ficar pronto..."+pc.describe_index(index_name).status)
    time.sleep(1)

index = pc.Index(index_name)

### Criar ou conectar ao VectorStore do Pinecone

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name,
)

## Construção da Cadeia RAG

#### Definir o prompt para a LLM

In [ ]:

prompt = ChatPromptTemplate.from_template("""Responda à pergunta com base apenas no contexto fornecido.
Se você não souber a resposta, diga que não sabe, não tente inventar uma resposta.

Contexto: {context}

Pergunta: {input}""")

### Criar a cadeia de documentos

In [ ]:

document_chain = create_stuff_documents_chain(llm, prompt)



### Criar a cadeia de recuperação

In [ ]:

retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), document_chain)

## Testando o Sistema RAG

In [ ]:
# Função para fazer perguntas
def ask_question(question):
    response = retrieval_chain.invoke({"input": question})
    print(f"Pergunta: {question}")
    print(f"Resposta: {response['answer']}")
    

In [ ]:
#Exemplos de perguntas (use os prompts e perguntas de teste )

pergunta1 = "Quais são os principais benefícios da atividade física regular para a saúde cardiovascular?"

pergunta2 = "Explique o princípio da sobrecarga progressiva na musculação."

pergunta3 = "Qual a importância dos carboidratos na nutrição para atividade física?"

pergunta4 = "Cite três exercícios essenciais para membros superiores e seus principais músculos trabalhados."

pergunta5 = "O que é periodização do treinamento e quais são seus principais ciclos?"

pergunta6 = "Quais são as principais estratégias para prevenir lesões na musculação?"

In [ ]:
ask_question(pergunta3)

## Limpeza (Opcional)

In [ ]:
# Para deletar o índice do Pinecone (use com cautela!)
# pc.delete_index(index_name)